## pybind11 array and buffer interface: A numpy user’s perspective

![pybind11-logo](images/pybind11_logo.png)

**Dboy Liao**

[Medium](https://medium.com/@dboyliao)
[GitHub](https://github.com/dboyliao)
[LinkedIn](https://www.linkedin.com/in/yin-chen-liao-69967188/)
[CakeResume](https://www.cakeresume.com/dboyliao)

# The Numpy `ndarray`

In [1]:
import numpy as np

In [2]:
array = np.arange(16, dtype=np.int8).reshape(4, 4).copy()
array

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]], dtype=int8)

```python
array.shape
```

```
array.strides
```

```python
array.data
```

```python
array.base
```

In [3]:
np.lib.stride_tricks

<module 'numpy.lib.stride_tricks' from '/Users/dboyliao/.pyenv/versions/3.8.1/lib/python3.8/site-packages/numpy/lib/stride_tricks.py'>

![array-2d](images/array_2d.drawio.svg)

In [4]:
print(array)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]


![array-2d-flatten](images/array_2d_flatten.drawio.svg)

In [5]:
arr_flatten = array.ravel()
arr_flatten

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
      dtype=int8)

In [6]:
print("1 == 4 * 0 + 1 * 1:", 1 == 4*0 + 1*1)
print("6 == 4 * 1 + 1 * 2:", 6 == 4*1 + 1*2)

1 == 4 * 0 + 1 * 1: True
6 == 4 * 1 + 1 * 2: True


In [7]:
print("arr_flatten[1] == array[0, 1]:", arr_flatten[1] == array[0, 1])
print("arr_flatten[6] == array[1, 2]:", arr_flatten[6] == array[1, 2])

arr_flatten[1] == array[0, 1]: True
arr_flatten[6] == array[1, 2]: True


In [8]:
array.strides

(4, 1)

![array-2d-view](images/array_2d_view.drawio.svg)

In [9]:
arr3 = array.reshape(2, 2, 2, 2)
arr3

array([[[[ 0,  1],
         [ 2,  3]],

        [[ 4,  5],
         [ 6,  7]]],


       [[[ 8,  9],
         [10, 11]],

        [[12, 13],
         [14, 15]]]], dtype=int8)

In [10]:
arr3.strides

(8, 4, 2, 1)

In [11]:
print("11 == 8 * 1 + 4 * 0 + 2 * 1 + 1 * 1:", 11 == 8 * 1 + 4 * 0 + 2 * 1 + 1 * 1)

11 == 8 * 1 + 4 * 0 + 2 * 1 + 1 * 1: True


In [12]:
arr_flatten[11] == arr3[1, 0, 1, 1]

True

- linear offset: the offset of an element in the flattened array
- $\mathbf{arr}$: a m-dims array
    - strides: $(s_0, s_1, ..., s_m)$
    - shape: $(d_0, d_1, ..., d_m)$
- $e = \mathbf{arr}[i_0, i_1, ..., i_m]$, element in $\mathbf{arr}$
    - with linear offset $\text{offset}_e$

Then we have:

$$
    \text{offset}_e = \sum\limits_{j=0}^{m} s_j \cdot i_j
$$

![okey-then-what](images/okey_then_what.jpg)

## Broadcasting Rule: From Scratch

[Numpy Documentation](https://numpy.org/doc/stable/user/basics.broadcasting.html#general-broadcasting-rules)

In [13]:
arr1 = np.arange(16, dtype=np.int8).reshape(4, 2, 2)
arr2 = np.array([1, 2], dtype=np.int8)

In [14]:
arr1.shape

(4, 2, 2)

In [15]:
arr2.shape

(2,)

In [16]:
(arr1 + arr2).shape

(4, 2, 2)

In [17]:
arr2.strides

(1,)

In [18]:
# promote arr2: make it of the shape as arr1
arr2_promoted = np.lib.stride_tricks.as_strided(
    arr2,
    shape=arr1.shape,
    strides=(0, 0)+arr2.strides
)

In [19]:
arr2_promoted.shape

(4, 2, 2)

In [20]:
arr2_promoted

array([[[1, 2],
        [1, 2]],

       [[1, 2],
        [1, 2]],

       [[1, 2],
        [1, 2]],

       [[1, 2],
        [1, 2]]], dtype=int8)

In [21]:
np.alltrue(arr1 + arr2 == arr1 + arr2_promoted)

True

In [22]:
arr1_promoted_lib, arr2_promoted_lib = np.lib.stride_tricks.broadcast_arrays(arr1, arr2)

In [23]:
arr2_promoted_lib.strides

(0, 0, 1)

In [24]:
# arr2 is the owner of the underlying memory block
arr2.base is None

True

In [25]:
# arr2_promoted_lib is just a shared-memory view of arr2
arr2_promoted_lib.base is arr2

True

In [26]:
array.flatten().base is None

True

In [27]:
array.ravel().base is None

False

![okey-then-what](images/okey_then_what.jpg)

## Time Series: Sliding Window with Shared-Memory View

In [28]:
data = np.arange(20, dtype=np.int8)
data

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19], dtype=int8)

In [29]:
windows = np.lib.stride_tricks.as_strided(
    data,
    shape=(16, 5),
    strides=(1, 1)
)

In [30]:
windows

array([[ 0,  1,  2,  3,  4],
       [ 1,  2,  3,  4,  5],
       [ 2,  3,  4,  5,  6],
       [ 3,  4,  5,  6,  7],
       [ 4,  5,  6,  7,  8],
       [ 5,  6,  7,  8,  9],
       [ 6,  7,  8,  9, 10],
       [ 7,  8,  9, 10, 11],
       [ 8,  9, 10, 11, 12],
       [ 9, 10, 11, 12, 13],
       [10, 11, 12, 13, 14],
       [11, 12, 13, 14, 15],
       [12, 13, 14, 15, 16],
       [13, 14, 15, 16, 17],
       [14, 15, 16, 17, 18],
       [15, 16, 17, 18, 19]], dtype=int8)

In [31]:
X = windows[:, :4]
Y = windows[:, 4]

In [32]:
X

array([[ 0,  1,  2,  3],
       [ 1,  2,  3,  4],
       [ 2,  3,  4,  5],
       [ 3,  4,  5,  6],
       [ 4,  5,  6,  7],
       [ 5,  6,  7,  8],
       [ 6,  7,  8,  9],
       [ 7,  8,  9, 10],
       [ 8,  9, 10, 11],
       [ 9, 10, 11, 12],
       [10, 11, 12, 13],
       [11, 12, 13, 14],
       [12, 13, 14, 15],
       [13, 14, 15, 16],
       [14, 15, 16, 17],
       [15, 16, 17, 18]], dtype=int8)

In [33]:
Y

array([ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
      dtype=int8)

In [34]:
X.base is None

False

In [35]:
Y.base is None

False

In [36]:
# X = windows[:, :4]
X2 = windows[:, [0, 1, 2, 3]]

In [37]:
np.alltrue(X2 == X)

True

In [38]:
X2.base is None

False

- Python Data Model: **\_\_getitem\_\_**
    - [link](https://docs.python.org/3/reference/datamodel.html#object.__getitem__)

- `start:end:step` is a short-hand for `slice` object
    - has **FIXED** pattern (fixed start, fixed end, fixed step)
    - can be convert to a set of corresponding strides
    - shared-memory view is possible
    - **NO COPY REQUIRED**

- a list of indices
    - no guarantee on the pattern of the indices contained
    - cannot be convert to a set of corresponding strides
    - **COPY IS A MUST**

![okey-then-what](images/okey_then_what.jpg)

# `pybind11` Buffer and Array Interface

[Documentation](https://pybind11.readthedocs.io/en/stable/advanced/pycpp/numpy.html#numpy)

- Good tutorial to CPython buffer protocol
    - https://jakevdp.github.io/blog/2014/05/05/introduction-to-the-python-buffer-protocol/
    - CPython Bufferinfo object: https://docs.python.org/3/c-api/buffer.html#buffer-structure
    - **WARN**: long and tedious C code

- `pybind11` make it easy to expose your type in C++ to Python via buffer protocol

## `pybind11::buffer_info`

```cpp
struct buffer_info {
    void *ptr;
    py::ssize_t itemsize;
    std::string format;
    py::ssize_t ndim;
    std::vector<py::ssize_t> shape;
    std::vector<py::ssize_t> strides;
};
```

```cpp
py::class_<MyMatrix>(m, "MyMatrix", py::buffer_protocol())
  .def(py::init<int, int>(), py::arg("rows"), py::arg("cols"))
  .def_buffer([](const MyMatrix &m) {
    return py::buffer_info(
        /* the memory block */
        m.data(),
        /* Size of one scalar */
        sizeof(float),
        /* string, Python struct-style format descriptor */
        py::format_descriptor<float>::format(),
        /* Number of dimensions */
        2,
        /* Buffer shape */
        {m.rows(), m.cols()},
        /* Strides (in bytes) for each index */
        {sizeof(float) * m.cols(), sizeof(float)});
  });
```

```python
def create_mat(rows, cols):
    return np.array(MyMatrix(rows, cols), copy=False)
```

In [39]:
import mylib

mylib.create_mat(5, 5)

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]], dtype=float32)

## `pybind11::array` and `pybind11::array_`

```cpp
py::array_t<double> array_add_value(const py::array_t<double> &in_arr,
                                    double v = 1.0) {
  auto info = in_arr.request();
  double *out_buff = new double[info.size];
  auto in_ptr = (double *)info.ptr;
  for (py::ssize_t idx = 0; idx < info.size; ++idx) {
    *(out_buff + idx) = *(in_ptr + idx) + v;
  }
  auto out_info = py::buffer_info(out_buff, info.itemsize, info.format,
                                  info.ndim, info.shape, info.strides);
  return py::array_t<double>(out_info);
}
```

In [40]:
import mylib

print(mylib.array_add_value.__doc__)

array_add_value(input_array: numpy.ndarray[numpy.float64], value: float = 0.0) -> numpy.ndarray[numpy.float64]

add by given value



In [41]:
arr_in = np.random.rand(5)
mylib.array_add_value(arr_in, 3.1)

array([3.36197318, 3.87817641, 3.71891186, 3.56103817, 3.87309472])

Let's change from `py::array_<T>` to `py::array`!

```cpp
py::array array_add_value(const py::array &in_arr, double v = 1.0) {
  auto info = in_arr.request();
  double *out_buff = new double[info.size];
  auto in_ptr = (double *)info.ptr;
  for (py::ssize_t idx = 0; idx < info.size; ++idx) {
    *(out_buff + idx) = *(in_ptr + idx) + v;
  }
  auto out_info = py::buffer_info(out_buff, info.itemsize, info.format,
                                  info.ndim, info.shape, info.strides);
  return py::array(out_info);
}
```

In [1]:
import mylib

print(mylib.array_add_value.__doc__)

array_add_value(input_array: numpy.ndarray, value: float = 0.0) -> numpy.ndarray

add by given value



In [2]:
arr_in = np.random.rand(5)
mylib.array_add_value(arr_in, 3.1)

array([3.64004604, 3.33775629, 3.14665428, 3.20065878, 3.7822936 ])

In [3]:
mylib.array_add_value(arr_in.astype(np.float128), 3.1)

array([              nan, -1.13644336e+0987,  1.68195440e-4932,
                     nan,  0.00000000e+0000], dtype=float128)

# Refereces

- buffer protocol
    - [test_buffers.cpp](https://github.com/pybind/pybind11/blob/ffa346860b306c9bbfb341aed9c14c067751feb8/tests/test_buffers.cpp)
    - [test_buffers.py](https://github.com/pybind/pybind11/blob/ffa346860b306c9bbfb341aed9c14c067751feb8/tests/test_buffers.py)